# Multi-class Image Classification using EfficientNet on Tiny ImageNet

In [1]:
!pip install tensorflow scikit-learn --quiet

In [ ]:
import os
import shutil
import requests
import zipfile

def download_and_extract():
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    filename = "tiny-imagenet-200.zip"

    if not os.path.exists(filename):
        print("Downloading Tiny ImageNet...")
        r = requests.get(url, stream=True)
        with open(filename, "wb") as f:
            shutil.copyfileobj(r.raw, f)
    else:
        print("Already downloaded.")

    if not os.path.exists("tiny-imagenet-200"):
        print("Extracting...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(".")
    else:
        print("Already extracted.")

def organize_val_images():
    val_dir = "tiny-imagenet-200/val"
    val_annotations_file = os.path.join(val_dir, "val_annotations.txt")
    val_images_dir = os.path.join(val_dir, "images")

    print("Organizing validation images...")
    # Read class annotations
    with open(val_annotations_file, "r") as f:
        lines = f.readlines()

    annotations = {}
    for line in lines:
        tokens = line.split("\t")
        img_file = tokens[0]
        class_id = tokens[1]
        annotations[img_file] = class_id

    # Create class folders
    for img_file, class_id in annotations.items():
        class_dir = os.path.join(val_dir, class_id)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        src = os.path.join(val_images_dir, img_file)
        dst = os.path.join(class_dir, img_file)
        shutil.move(src, dst)

    # Cleanup
    shutil.rmtree(val_images_dir)
    print("Validation images organized.")

download_and_extract()
organize_val_images()

### Import Required Libraries
- TensorFlow and Keras modules for modeling, preprocessing, and training.
- `EfficientNetB0` for loading a pretrained backbone.
- Callbacks for early stopping and dynamic learning rate scheduling.

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import CategoricalCrossentropy
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

### Set Paths and Hyperparameters
- `IMAGE_SIZE`: 64x64 as required by Tiny ImageNet.
- `NUM_CLASSES`: 200 categories in Tiny ImageNet.
- Directories for train and validation folders should be properly structured with subfolders per class.


In [23]:
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 64
NUM_CLASSES = 200
EPOCHS = 3
TRAIN_DIR = 'tiny-imagenet-200/train'
VAL_DIR = 'tiny-imagenet-200/val'
TEST_DIR = 'tiny-imagenet-200/test'

### Data Augmentation and Preprocessing
- `ImageDataGenerator` is used to apply real-time data augmentation.
- Includes random rotation, zoom, shear, shift, flip, etc.
- Helps reduce overfitting and makes the model more robust.

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

### Load the Dataset with Flow from Directory
- Training and validation generators load images from directories.
- Images are resized to 64x64 and labels are one-hot encoded (`categorical` mode).
- Batch size is set for efficient training.

In [8]:
train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


### Load Pretrained EfficientNetB0
- The EfficientNetB0 backbone is loaded without the top dense layers (`include_top=False`).
- Pretrained on ImageNet.
- Input size adjusted to Tiny ImageNet size.
- `trainable=False` to freeze its weights initially.

In [24]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
base_model.trainable = False  # Freeze base model for feature extraction

### Add Custom Classification Head
- Add `GlobalAveragePooling2D` to reduce spatial dimensions.
- Add dense layer with ReLU and Dropout.
- Add softmax output layer for 200-class prediction.

In [25]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

<img src="https://www.researchgate.net/publication/360334595/figure/fig2/AS:11431281091326457@1666379680352/EfficientNetB0-Network-Architecture.ppm">

<img src="https://i.sstatic.net/wjMIUl0Y.png">

In [26]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 64, 64, 3) │          0 │ input_layer_2[0]… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 64, 64, 3) │          7 │ rescaling_4[0][0] │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_5         │ (None, 64, 64, 3) │          0 │ normalization_2[… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 65, 65, 3) │          0 │ rescaling_5[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 32, 32,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 32, 32,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 32, 32,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 32, 32,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 32, 32,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 32, 32,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 32, 32,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 32, 32,    │        512 │ block1a_se_excit

 Total params: 4,808,043 (18.34 MB)

 Trainable params: 758,472 (2.89 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

### Compile the Model with Label Smoothing
- Optimizer: Adam with learning rate 0.001.
- Loss: `CategoricalCrossentropy` with `label_smoothing=0.1` to prevent overconfidence.
- Metric: Accuracy.

In [27]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

### Callbacks for Robust Training
- `EarlyStopping`: Stops training if validation loss doesn't improve.
- `ReduceLROnPlateau`: Reduces learning rate if the model stalls.

In [28]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3)
]

### Train the Model
- Run training with validation.
- Use fit() with the generators and callbacks.

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/3
 466/1563 ━━━━━━━━━━━━━━━━━━━━ 2:20 128ms/step - accuracy: 0.0050 - loss: 5.3199

### Unfreeze and Fine-tune the Base Model
- Enable training on the base model for transfer learning.
- Recompile with a smaller learning rate.
- Further train for additional epochs.

In [ ]:
### Unfreeze Some Layers for Fine-Tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer=Adam(1e-5),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

### Continue Training (Fine-Tuning)
- Resume training with a few unfrozen layers to boost performance.

In [ ]:
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=2,
    callbacks=callbacks
)

### Evaluate the Model on Test Set
- Report final accuracy on test data.

In [ ]:
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
y_true = []
y_pred = []

for images, labels in val_gen:
    probs = model.predict(images)
    preds = np.argmax(probs, axis=1)
    y_pred.extend(preds)
    y_true.extend(np.argmax(labels.numpy(), axis=1))

### Generate Classification Report
- Predict test labels.
- Use `classification_report()` to show precision, recall, and F1-score.

In [ ]:
print(classification_report(y_true, y_pred))

### Plot Confusion Matrix
- Compute and visualize confusion matrix to analyze misclassifications.

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()